# DEMO sprint 2 - Deel 1 (belastingen)
Inlezen waterstanden uit verschillende bronsystemen:
- RWS Waterinfo
- FEWS
- (Noos) Matroos
- RWS WaterWebServices

Gebruikmakend van:
- csv bestanden voor uitwisselen van data

### Benodigde generieke Python packages

In [1]:
from pathlib import Path

### Definitie configuratie: CSV bestanden 

GlobalVariables:
    rootdir: "data_demo" 
    moments: [-24,0,24,48]  

    LoadsWaterinfo:
        parameters: ["waterhoogte"]
        MISSING_VALUE: 999

    LoadsFews:
        host: "https://fews.hhnk.nl"
        port: 443
        region: "fewspiservice"
        version: "1.25"
        filter: "HHNK_WEB"
        parameters: ["WATHTE [m][NAP][OW]"]
        MISSING_VALUE: 999
    
    LoadsMatroos:
        website: "vitaal"
        model: "dcsm6_kf"
        parameters: ["WATHTE"]
        MISSING_VALUE: 999
    
    LoadsWaterwebservicesRWS:
        parameters: ["WATHTE"]
        MISSING_VALUE: 999

DataAdapter: 
    default_options:
        csv_source:
            sep: ";"
        csv:
            sep: ","
    BelastingLocaties_WaterInfo: 
        type: csv_source
        filter: "WaterInfo"
        path: "locaties.csv"
    Waterstanden_WaterInfo: 
        type: csv
        path: "waterstanden_waterinfo.csv"
    BelastingLocaties_FEWS: 
        type: csv_source
        filter: "FEWS"
        path: "locaties.csv"
    Waterstanden_FEWS: 
        type: csv
        path: "waterstanden_fews.csv"
    BelastingLocaties_Matroos: 
        type: csv_source
        filter: "NOOS Matroos"
        path: "locaties.csv"
    Waterstanden_Matroos: 
        type: csv
        path: "waterstanden_matroos.csv"
    BelastingLocaties_Waterwebservices_RWS: 
        type: csv_source
        filter: "RWS Waterwebservices"
        path: "locaties.csv"
    Waterstanden_Waterwebservices_RWS: 
        type: csv
        path: "waterstanden_waterwebservices_rws.csv"

### Lees configuratie in en maak dictionary base.Config aan

In [17]:
from toolbox_continu_inzicht import Config

yaml_config_file = "demo_sprint2-I.yaml"
path = Path.joinpath(Path.cwd(), "data_demo", yaml_config_file)
c = Config(config_path=path)
c.lees_config()

### Geef config mee aan base.Adapter

In [18]:
from toolbox_continu_inzicht import DataAdapter

data = DataAdapter(config=c)
print(f"Folder met data: {data.config.config_path}")
print(f"Globale variabelen: {data.config.global_variables}")
for data_adapter in data.config.data_adapters:
    print(f"Data apdater: {data_adapter}")

Folder met data: c:\Data\Python\toolbox-continu-inzicht\tests\demos\loads\data_demo\demo_sprint2-I.yaml
Globale variabelen: {'rootdir': 'data_demo', 'moments': [-24, 0, 24, 48], 'LoadsWaterinfo': {'parameters': ['waterhoogte'], 'MISSING_VALUE': 999}, 'LoadsFews': {'host': 'https://fews.hhnk.nl', 'port': 443, 'region': 'fewspiservice', 'version': '1.25', 'filter': 'HHNK_WEB', 'parameters': ['WATHTE [m][NAP][OW]'], 'MISSING_VALUE': 999}, 'LoadsMatroos': {'website': 'vitaal', 'model': 'dcsm6_kf', 'parameters': ['WATHTE'], 'MISSING_VALUE': 999}, 'LoadsWaterwebservicesRWS': {'parameters': ['WATHTE'], 'MISSING_VALUE': 999}}
Data apdater: default_options
Data apdater: BelastingLocaties_WaterInfo
Data apdater: Waterstanden_WaterInfo
Data apdater: BelastingLocaties_FEWS
Data apdater: Waterstanden_FEWS
Data apdater: BelastingLocaties_Matroos
Data apdater: Waterstanden_Matroos
Data apdater: BelastingLocaties_Waterwebservices_RWS
Data apdater: Waterstanden_Waterwebservices_RWS


### Invoerbestand (CSV) met data van belastinglocaties
Verplichte velden (schema):
- measurement_location_id: int64
- measurement_location_code: object (string)
- measurement_location_description: object (string)

measurement_location_id;measurement_location_code;measurement_location_description;source;tide\
1;Pannerdense-kop(PANN);Pannerdense kop;WaterInfo;false\
2;MPN-N-24;Meetpunt Spijkerboor;FEWS;false\
3;hoekvanholland;Hoek van Holland;NOOS Matroos;true\
4;9889;Hintham beneden;RWS Waterwebservices;false

### Laad module (functie) LoadsWaterinfo voor inlezen belastingen uit RWS Waterinfo

In [19]:
from toolbox_continu_inzicht.loads import LoadsWaterinfo

waterinfo = LoadsWaterinfo(data_adapter=data)

### Run module (functie) LoadsWaterinfo voor inlezen waterstanden uit Waterinfo
LET OP: zoek op basis van een measurement_location_code die in RWS WaterInfo overeenkomt met 'locationCodes'!\
https://waterinfo.rws.nl/api/chart/get?mapType=waterhoogte&locationCodes=Pannerdense-kop(PANN)&values=-48%2C48

In [20]:
ds = waterinfo.run(input="BelastingLocaties_WaterInfo", output="Waterstanden_WaterInfo")

In [21]:
measuringlocations = waterinfo.df_in
measuringlocations

,measurement_location_id,measurement_location_code,measurement_location_description,source,tide
0,1,Pannerdense-kop(PANN),Pannerdense kop,WaterInfo,False


In [22]:
waterlevels = waterinfo.df_out
waterlevels

,measurement_location_id,measurement_location_code,measurement_location_description,parameter_id,parameter_code,parameter_description,unit,date_time,value,value_type
0,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,cm,2024-11-11 09:40:00+00:00,772.0,meting
1,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,cm,2024-11-11 09:50:00+00:00,772.0,meting
2,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,cm,2024-11-11 10:00:00+00:00,771.0,meting
3,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,cm,2024-11-11 10:10:00+00:00,772.0,meting
4,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,cm,2024-11-11 10:20:00+00:00,773.0,meting
...,...,...,...,...,...,...,...,...,...,...
571,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte verwacht Oppervlaktewater t.o.v. N...,cm,2024-11-15 05:20:00+00:00,757.0,verwachting
572,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte verwacht Oppervlaktewater t.o.v. N...,cm,2024-11-15 05:30:00+00:00,757.0,verwachting
573,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte verwacht Oppervlaktewater t.o.v. N...,cm,2024-11-15 05:40:00+00:00,757.0,verwachting
574,1,Pannerdense-kop(PANN),Pannerdense kop,4724,WATHTE,Waterhoogte verwacht Oppervlaktewater t.o.v. N...,cm,2024-11-15 05:50:00+00:00,757.0,verwachting


### Laad en run module (functie) LoadsFEWS voor inlezen belastingen uit FEWS
LET OP: zoek op basis van een measurement_location_code die in FEWS overeenkomt met 'locationId'!\
https://fews.hhnk.nl/FewsWebServices/rest/fewspiservice/v1/locations\
![image info](data_demo/images/FEWS_locationIs.PNG)

In [23]:
from toolbox_continu_inzicht.loads import LoadsFews

fews = LoadsFews(data_adapter=data)
ds = fews.run(input="BelastingLocaties_FEWS", output="Waterstanden_FEWS")

In [24]:
measuringlocations = fews.df_in
measuringlocations

,measurement_location_id,measurement_location_code,measurement_location_description,source,tide
1,2,MPN-N-24,Meetpunt Spijkerboor,FEWS,False


In [25]:
waterlevels = fews.df_out
waterlevels

,measurement_location_id,measurement_location_code,measurement_location_description,parameter_id,parameter_code,parameter_description,unit,date_time,value,value_type
0,2,MPN-N-24,Meetpunt Spijkerboor,4724,WATHTE [m][NAP][OW],WATHTE [m][NAP][OW],m,2024-11-12 09:00:00+00:00,-0.463,meting
1,2,MPN-N-24,Meetpunt Spijkerboor,4724,WATHTE [m][NAP][OW],WATHTE [m][NAP][OW],m,2024-11-12 09:15:00+00:00,-0.463,meting
2,2,MPN-N-24,Meetpunt Spijkerboor,4724,WATHTE [m][NAP][OW],WATHTE [m][NAP][OW],m,2024-11-12 09:30:00+00:00,-0.475,meting
3,2,MPN-N-24,Meetpunt Spijkerboor,4724,WATHTE [m][NAP][OW],WATHTE [m][NAP][OW],m,2024-11-12 09:45:00+00:00,-0.482,meting
4,2,MPN-N-24,Meetpunt Spijkerboor,4724,WATHTE [m][NAP][OW],WATHTE [m][NAP][OW],m,2024-11-12 10:00:00+00:00,-0.472,meting
...,...,...,...,...,...,...,...,...,...,...
93,2,MPN-N-24,Meetpunt Spijkerboor,4724,WATHTE [m][NAP][OW],WATHTE [m][NAP][OW],m,2024-11-13 08:15:00+00:00,-0.479,meting
94,2,MPN-N-24,Meetpunt Spijkerboor,4724,WATHTE [m][NAP][OW],WATHTE [m][NAP][OW],m,2024-11-13 08:30:00+00:00,-0.483,meting
95,2,MPN-N-24,Meetpunt Spijkerboor,4724,WATHTE [m][NAP][OW],WATHTE [m][NAP][OW],m,2024-11-13 08:45:00+00:00,-0.482,meting
96,2,MPN-N-24,Meetpunt Spijkerboor,4724,WATHTE [m][NAP][OW],WATHTE [m][NAP][OW],m,2024-11-13 09:00:00+00:00,-0.474,meting


### Laad en run module (functie) LoadsMatroos voor inlezen belastingen uit Matroos
LET OP: zoek op basis van een measurement_location_code die in (NOOS) Matroos overeenkomt met 'Location' -> loc_id!\
https://vitaal.matroos.rws.nl/matroos/php/image_series.php?type=image&source_id=156&loc_id=7&unit_id=1&colors=blue&ymin=0&ymax=0&label=0&alarm=0&alert=0&prealert=0&localtime_offset=0&tcurrent=2024-11-11%2019:38:11&tstart=2024-11-10%2000:00:00&tstop=2024-11-13%2000:00:00&anal_time=2024-11-11%2000:00:00

In [26]:
from toolbox_continu_inzicht.loads import LoadsMatroos

matroos = LoadsMatroos(data_adapter=data)
ds = matroos.run(input="BelastingLocaties_Matroos", output="Waterstanden_Matroos")

In [27]:
measuringlocations = matroos.df_in
measuringlocations

,measurement_location_id,measurement_location_code,measurement_location_description,source,tide
2,3,hoekvanholland,Hoek van Holland,NOOS Matroos,True


In [28]:
waterlevels = matroos.df_out
waterlevels

,measurement_location_id,measurement_location_code,measurement_location_description,parameter_id,parameter_code,date_time,unit,value,value_type
0,7,hoekvanholland,Hoek van Holland,4724,WATHTE,2024-11-12 09:00:00+00:00,cm,3.95,meting
1,7,hoekvanholland,Hoek van Holland,4724,WATHTE,2024-11-12 09:10:00+00:00,cm,11.30,meting
2,7,hoekvanholland,Hoek van Holland,4724,WATHTE,2024-11-12 09:20:00+00:00,cm,19.50,meting
3,7,hoekvanholland,Hoek van Holland,4724,WATHTE,2024-11-12 09:30:00+00:00,cm,29.61,meting
4,7,hoekvanholland,Hoek van Holland,4724,WATHTE,2024-11-12 09:40:00+00:00,cm,39.41,meting
...,...,...,...,...,...,...,...,...,...
410,7,hoekvanholland,Hoek van Holland,4724,WATHTE,2024-11-15 05:20:00+00:00,cm,-40.88,verwachting
411,7,hoekvanholland,Hoek van Holland,4724,WATHTE,2024-11-15 05:30:00+00:00,cm,-42.46,verwachting
412,7,hoekvanholland,Hoek van Holland,4724,WATHTE,2024-11-15 05:40:00+00:00,cm,-41.99,verwachting
413,7,hoekvanholland,Hoek van Holland,4724,WATHTE,2024-11-15 05:50:00+00:00,cm,-39.54,verwachting


### Laad en run module (functie) LoadsWaterwebservicesRWS voor inlezen belastingen uit RWS Waterwebservices
LET OP: zoek op basis van een measurement_location_code die in RWS WaterWebservices overeenkomt met Locatie_MessageID!\
![image info](data_demo/images/WaterWebservicesRWS_Locatie_MessageId.PNG)

In [29]:
from toolbox_continu_inzicht.loads import LoadsWaterwebservicesRWS

waterwebservices_rws = LoadsWaterwebservicesRWS(data_adapter=data)
ds = waterwebservices_rws.run(
    input="BelastingLocaties_Waterwebservices_RWS",
    output="Waterstanden_Waterwebservices_RWS",
)

In [30]:
measuringlocations = waterwebservices_rws.df_in
measuringlocations

,measurement_location_id,measurement_location_code,measurement_location_description,source,tide
3,4,9889,Hintham beneden,RWS Waterwebservices,False


In [31]:
waterlevels = waterwebservices_rws.df_out
waterlevels

,measurement_location_id,measurement_location_code,measurement_location_description,parameter_id,parameter_code,date_time,unit,value,value_type
0,4,9889,Hintham beneden,4724,WATHTE,2024-11-12 10:00:00+01:00,cm,199.0,meting
1,4,9889,Hintham beneden,4724,WATHTE,2024-11-12 10:10:00+01:00,cm,191.0,meting
2,4,9889,Hintham beneden,4724,WATHTE,2024-11-12 10:20:00+01:00,cm,197.0,meting
3,4,9889,Hintham beneden,4724,WATHTE,2024-11-12 10:30:00+01:00,cm,201.0,meting
4,4,9889,Hintham beneden,4724,WATHTE,2024-11-12 10:40:00+01:00,cm,193.0,meting
...,...,...,...,...,...,...,...,...,...
142,4,9889,Hintham beneden,4724,WATHTE,2024-11-13 09:40:00+01:00,cm,200.0,meting
143,4,9889,Hintham beneden,4724,WATHTE,2024-11-13 09:50:00+01:00,cm,205.0,meting
144,4,9889,Hintham beneden,4724,WATHTE,2024-11-13 10:00:00+01:00,cm,200.0,meting
145,4,9889,Hintham beneden,4724,WATHTE,2024-11-13 10:10:00+01:00,cm,202.0,verwachting


### Grafische weergave van resultaten in 4 grafieken